In [1]:
import sys
import os
import pickle

from importlib_metadata.compat.py39 import ep_matches
from omegaconf import DictConfig, OmegaConf
from pathlib import Path

In [2]:
# # 将 MScProject 目录添加到 Python 模块搜索路径
# sys.path.append(os.path.abspath('..'))
# 从 DataGenerate.py 导入 DataGenerate 类 用于pickle导入
from DataInit import DataManager, RewardDataManager


In [3]:
# 加载 config.yaml 文件
config = OmegaConf.load("config/config.yaml")
global_path = Path(config.path.global_path)
data_path = global_path / 'Data'
# 打印完整的配置内容
print(f'---------- Config Info ----------')
print(OmegaConf.to_yaml(config))
# 打印全局路径和数据路径
print(f'---------- Path Info ----------')
print(f'Global Path: {global_path}')
print(f'Data Path: {data_path}')

---------- Config Info ----------
path:
  global_path: E:/Study in the UK/Project/MScProject
base:
  'N': 10
  T: 11000
  T_train_val: 10000
  train_ratio: 0.8
  T_train: 8000
  T_val: 2000
  T_test: 1000
  lambda_load: 0.5
  top_k:
  - 1
  - 2
  - 3
  - 4
  - 5
data_generation:
  load_data:
    node_load_mean_mean: 50.0
    node_load_mean_var: 10.0
    node_load_iid_var: 5.0
    node_load_ar1_theta: 0.9
  latency_data:
    node_latency_mean_mean: 30.0
    node_latency_mean_var: 10.0
    node_latency_ar1_theta: 0.9
  reward_parameters:
    iid:
      alpha_load_0: 40.0
      alpha_latency_1: 0.041
    ar1:
      alpha_load_0: 40.0
      alpha_latency_1: 0.041
  reward_parameters_slider:
    alpha_load_0:
      value: 1.0
      min: 0.001
      max: 40.0
      step: 0.01
      description: alpha_load_0
    alpha_latency_0:
      value: 1.0
      min: 0.001
      max: 6.0
      step: 0.01
      description: alpha_latency_0
    alpha_latency_1:
      value: 0.5
      min: 0.0001
      max

In [4]:
def import_data_manager(data_type: str) -> DataManager:
    """
    加载之前保存的数据管理对象。

    :param data_path: 数据保存的路径
    :param data_type: 数据类型，例如 'iid_load', 'ar1_load', 'iid_latency', 'ar1_latency'
    :return: 加载的 DataManager 对象
    """
    file_path = data_path / f'{data_type}_data_manage.pkl'

    with open(file_path, 'rb') as f:
        data_manager = pickle.load(f)

    return data_manager

# 示例调用
load_iid_data_manage = import_data_manager('iid_load')
load_ar1_data_manage = import_data_manager('ar1_load')
latency_iid_data_manage = import_data_manager('iid_latency')
latency_ar1_data_manage = import_data_manager('ar1_latency')


C:\Users\Lenovo\AppData\Roaming\Python\Python311\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


In [5]:
# # 绘制数据
# load_iid_data_manage.plot_range_data(load_iid_data_manage.data_np[:3, :], title='Load IID Data')
# load_ar1_data_manage.plot_range_data(load_ar1_data_manage.data_np[:3, :], title='Load AR(1) Data')
# latency_iid_data_manage.plot_range_data(latency_iid_data_manage.data_np[:3, :], title='Latency IID Data')
# latency_ar1_data_manage.plot_range_data(latency_ar1_data_manage.data_np[:3, :], title='Latency AR(1) Data')

In [6]:
with open(data_path/'reward_data_manager.pkl', 'rb') as f:
    reward_data_manager = pickle.load(f)

In [7]:
# reward_data_manager.print_info()

In [8]:
# reward_data_manager.plot_reward_data()

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import os
import re
from scipy.ndimage import uniform_filter1d
from scipy.signal import savgol_filter
from scipy.stats import norm

In [10]:
from scipy.ndimage import gaussian_filter1d
def moving_average(data, window_size=50):
    """简单滑动平均平滑"""
    return np.convolve(data, np.ones(window_size)/window_size, mode='same')

def exponential_moving_average(data, alpha=0.01):
    """指数加权平均平滑"""
    smoothed_data = np.zeros_like(data)
    smoothed_data[0] = data[0]
    for t in range(1, len(data)):
        smoothed_data[t] = alpha * data[t] + (1 - alpha) * smoothed_data[t-1]
    return smoothed_data

def gaussian_smooth(data, sigma=15):
    """高斯平滑"""
    return gaussian_filter1d(data, sigma=sigma)

In [11]:
import ipywidgets as widgets
from ipywidgets import interact, widgets, HBox, VBox, Output, interactive_output, interactive, GridBox, Layout
from IPython.display import display

In [50]:
class MAB:
    def __init__(self, config, reward_data_manager):
        self.config = config
        self.reward_data_manager = reward_data_manager

        # 初始化通用参数
        self.lambda_load = self.config.base.lambda_load
        self.top_k = self.config.base.top_k
        self.N = self.config.base.N
        self.T_test = self.config.base.T_test

        # 加载数据
        self.iid_load = reward_data_manager.iid_load
        self.iid_load_reward_0 = reward_data_manager.iid_load_reward_0
        self.iid_load_reward_1 = reward_data_manager.iid_load_reward_1
        self.iid_latency = reward_data_manager.iid_latency
        self.iid_latency_reward_1 = reward_data_manager.iid_latency_reward_1
        self.ar1_load = reward_data_manager.ar1_load
        self.ar1_load_reward_0 = reward_data_manager.ar1_load_reward_0
        self.ar1_load_reward_1 = reward_data_manager.ar1_load_reward_1
        self.ar1_latency = reward_data_manager.ar1_latency
        self.ar1_latency_reward_1 = reward_data_manager.ar1_latency_reward_1

        # 定义一个映射字典
        self.reward_mapping = {
            ('iid', 'reward_0'): (self.iid_load_reward_0, self.iid_latency_reward_1),
            ('iid', 'reward_1'): (self.iid_load_reward_1, self.iid_latency_reward_1),
            ('ar1', 'reward_0'): (self.ar1_load_reward_0, self.ar1_latency_reward_1),
            ('ar1', 'reward_1'): (self.ar1_load_reward_1, self.ar1_latency_reward_1),
        }
        
        self.load_reward_method = None
        self.data_type = None
        self.load_reward, self.latency_reward = None, None
        self.combine_reward = None
        self.combine_reward_mean = None
        self.combine_reward_optimal_node = None
        self.combine_reward_optimal_mean = None
        self.combine_reward_sorted_mean = None

        self.algorithm = None

    def set_parameters(self, load_reward_method: str, data_type: str) -> None:
        """
        设置参数并计算组合奖励。
        :param load_reward_method: 加载奖励的方法，'reward_0' 或 'reward_1'
        :param data_type: 数据类型，'iid' 或 'ar1'
        """
        # 重新设置参数
        self.load_reward_method = load_reward_method
        self.data_type = data_type
        try:
            self.load_reward, self.latency_reward = self.reward_mapping[(data_type, load_reward_method)]
        except KeyError:
            raise ValueError(f'Invalid load_reward_method: {load_reward_method}, data_type: {data_type}')

        # 计算组合奖励
        self.combine_reward = self.lambda_load * self.load_reward + (1 - self.lambda_load) * self.latency_reward
        self.combine_reward_mean = np.mean(self.combine_reward, axis=1)
        self.combine_reward_optimal_node = np.argmax(self.combine_reward_mean)
        self.combine_reward_optimal_mean = np.max(self.combine_reward_mean)
        self.combine_reward_sorted_mean = np.argsort(self.combine_reward_mean)[::-1]

    def calculate_top_k_accuracy(self, time_counts: np.ndarray, k_list: list) -> dict:
        """
        计算 MAB 过程的 Top-k Accuracy（定义二）。
        
        参数:
        - time_counts: 一个数组，表示在每个时间步中选择的节点。
        - k_list: 一个整数列表，表示需要计算的 k 值。
        
        返回:
        - 一个字典，key 是 k 的值，value 是对应的 Top-k Accuracy。
        
        说明:
        - Top-k Accuracy 的定义二：在 T_test 个时间步中，选择的节点中有多少是最优节点的前 k 个节点。
        """
        
        top_k_accuracy = {}
        T = len(time_counts)

        for k in k_list:
            correct_count = 0
            optimal_nodes = self.combine_reward_sorted_mean[:k]  # 选择前 k 个最佳节点
            for t in range(T):
                if time_counts[t] in optimal_nodes:
                    correct_count += 1
            accuracy = correct_count / T
            top_k_accuracy[k] = accuracy

        return top_k_accuracy

    def plot_combine_rewards(self, selected_nodes=None) -> None:
        """
        绘制组合奖励的图形。
        
        参数:
        - selected_nodes: 一个列表，表示选择的节点的索引。默认为 None，表示选择前 3 个节点。
        
        说明:
        - 组合奖励是加载奖励和延迟奖励的加权和。
        """
        
        if selected_nodes is None:
            selected_nodes = [0, 1, 2]  # 默认选择前3个节点
    
        # 数据准备
        last_T_data = self.combine_reward[selected_nodes, -self.T_test:]
    
        # 使用 constrained_layout 进行更智能的布局管理
        fig, axs = plt.subplots(1, 3, figsize=(12, 4), constrained_layout=True)
    
        # 1. combine_reward中选择节点的最后T_test次数据的展示
        for i, node in enumerate(selected_nodes):
            axs[0].plot(last_T_data[i], label=f'Node {node}')
        axs[0].set_title('Last T_test Data - Selected Nodes')
        axs[0].set_xlabel('Time')
        axs[0].set_ylabel('Value')
        axs[0].legend()
    
        # 2. N个节点的均值的分布图，并标注最值
        means = self.combine_reward_mean
        axs[1].plot(means, 'bo-', label='Mean Value per Node')
        axs[1].axhline(y=np.min(means), color='red', linestyle='--', label='Range')
        axs[1].axhline(y=np.max(means), color='red', linestyle='--')
        axs[1].set_title('Mean Values of Nodes')
        axs[1].set_xlabel('Node')
        axs[1].set_ylabel('Mean Value')
        axs[1].legend()
        
        # 调整标注的位置，避免超出图形范围
        axs[1].annotate(f'Min: {np.min(means):.3f}', xy=(np.argmin(means), np.min(means)), 
                        xytext=(np.argmin(means) + 0.5, np.min(means) - 0.05),
                        arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=10, color='black')
        axs[1].annotate(f'Max: {np.max(means):.3f}', xy=(np.argmax(means), np.max(means)), 
                        xytext=(np.argmax(means) + 0.5, np.max(means) + 0.05),
                        arrowprops=dict(facecolor='black', arrowstyle='->'), fontsize=10, color='black')
    
        # 3. 选择节点的最后T_test次数据的直方图
        for i, node in enumerate(selected_nodes):
            axs[2].hist(last_T_data[i], alpha=0.5, label=f'Node {node}')
        axs[2].set_title('Histogram - Selected Nodes')
        axs[2].set_xlabel('Value')
        axs[2].set_ylabel('Frequency')
        axs[2].legend()
    
        plt.show()
        
    class EpsilonGreedy:
        """
        Epsilon-Greedy 算法类。
        这是最初级的多臂老虎机算法，根据 epsilon 的值以一定概率随机选择节点，以 1-epsilon 的概率选择当前平均奖励最高的节点。
        
        功能：
        - algorithm: 运行 epsilon-greedy 算法，返回选择的节点、每个节点的选择次数、单次遗憾、累积遗憾以及 Top-k Accuracy。
        - plot_results: 绘制 epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾以及 Top-k Accuracy，并应用平滑。
        - plot_epsilon_vs_cumulative_regret: 绘制 epsilon vs cumulative regret 图。
        - dynamic_plot: 动态可视化 epsilon-greedy 算法的结果，epsilon 可以通过滑块调整。
        - run_and_plot: 运行 epsilon-greedy 算法并绘制结果，用于动态可视化图中的 epsilon 可以通过滑块调整。
        """
        def __init__(self, mab):
            self.mab = mab
            self.config_eg = self.mab.config.epsilon_greedy

            self.dynamic_epsilon_min = self.config_eg.dynamic_plot.epsilon_min  # 动态 epsilon 的最小值
            self.dynamic_epsilon_max = self.config_eg.dynamic_plot.epsilon_max  # 动态 epsilon 的最大值
            self.dynamic_epsilon_step = self.config_eg.dynamic_plot.epsilon_step  # 动态 epsilon 的步长
            self.dynamic_epsilon_default_value = self.config_eg.dynamic_plot.epsilon_default_value  # 动态 epsilon 的默认值
            
            self.e_vs_cr_epsilon_min = self.config_eg.epsilon_vs_cumulative_regret.epsilon_min  # epsilon vs cumulative regret 图的 epsilon 最小值
            self.e_vs_cr_epsilon_max = self.config_eg.epsilon_vs_cumulative_regret.epsilon_max  # epsilon vs cumulative regret 图的 epsilon 最大值
            self.e_vs_cr_epsilon_step = self.config_eg.epsilon_vs_cumulative_regret.epsilon_step  # epsilon vs cumulative regret 图的 epsilon 步长

        def algorithm(self, epsilon):
            def choose_node(epsilon, estimated_means):
                if np.random.rand() < epsilon:
                    return np.random.randint(self.mab.N)
                else:
                    return np.argmax(estimated_means)

            estimated_means = np.zeros(self.mab.N)
            time_counts = np.zeros(self.mab.T_test)
            nodes_counts = np.zeros(self.mab.N)
            single_step_regret = np.zeros(self.mab.T_test)

            for t in range(self.mab.T_test):
                chosen_node = choose_node(epsilon, estimated_means)
                time_counts[t] = chosen_node
                nodes_counts[chosen_node] += 1
                reward = self.mab.combine_reward[chosen_node, t]
                estimated_means[chosen_node] += (reward - estimated_means[chosen_node]) / nodes_counts[chosen_node]
                single_step_regret[t] = self.mab.combine_reward_optimal_mean - reward

            cumulative_regret = np.cumsum(single_step_regret)
            top_k_accuracy = self.mab.calculate_top_k_accuracy(time_counts, self.mab.top_k)
            return time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy

        def plot_results(self, time_counts: np.ndarray, single_step_regret: np.ndarray, cumulative_regret: np.ndarray, top_k_accuracy: dict, epsilon: float) -> None:
            """
            绘制 epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾以及 Top-k Accuracy，并应用平滑。
            
            参数:
            - time_counts: 一个数组，表示在每个时间步中选择的节点。
            - single_step_regret: 一个数组，表示每次选择的单次遗憾。
            - cumulative_regret: 一个数组，表示累积遗憾。
            - top_k_accuracy: 一个字典，表示不同 k 值对应的 Top-k Accuracy
            - epsilon: 当前实验使用的 epsilon 值。
            """
            # 应用平滑方法
            smoothed_single_step_regret = exponential_moving_average(single_step_regret)
            smoothed_cumulative_regret = exponential_moving_average(cumulative_regret)

            fig, axs = plt.subplots(2, 2, figsize=(14, 10))
            
            fig.suptitle(f"Epsilon-Greedy Results\nEpsilon: {epsilon}, Load Reward Method: {self.mab.load_reward_method}, Data Type: {self.mab.data_type}", fontsize=16)

            # 子图1：time_counts 绘制 T_test 次里每次选择的节点
            axs[0, 0].plot(time_counts, marker='o', linestyle='-', color='blue')
            axs[0, 0].set_title('Node Selection Over Time')
            axs[0, 0].set_xlabel('Time Step')
            axs[0, 0].set_ylabel('Chosen Node')

            # 在子图1上绘制 combine_reward_optimal_node 的横向虚线
            optimal_node = self.mab.combine_reward_optimal_node
            axs[0, 0].axhline(y=optimal_node, color='red', linestyle='--', label='Optimal Node')
            axs[0, 0].legend()

            # 子图2：单次遗憾的平滑曲线
            axs[0, 1].plot(smoothed_single_step_regret, marker='o', linestyle='-', color='green', label='Smoothed Single Step Regret')
            axs[0, 1].set_title('Single Step Regret (Smoothed)')
            axs[0, 1].set_xlabel('Time Step')
            axs[0, 1].set_ylabel('Regret')

            # 标记最大值和最小值
            min_idx = np.argmin(smoothed_single_step_regret)
            axs[0, 1].annotate(f'Min (x={min_idx}, y={smoothed_single_step_regret[min_idx]:.2f})',
                               xy=(min_idx, smoothed_single_step_regret[min_idx]),
                               xytext=(min_idx, smoothed_single_step_regret[min_idx] + 0.05),
                               arrowprops=dict(facecolor='blue', arrowstyle='->'),
                               fontsize=10, color='blue')

            max_idx = np.argmax(smoothed_single_step_regret)
            axs[0, 1].annotate(f'Max (x={max_idx}, y={smoothed_single_step_regret[max_idx]:.2f})',
                               xy=(max_idx, smoothed_single_step_regret[max_idx]),
                               xytext=(max_idx, smoothed_single_step_regret[max_idx] - 0.05),
                               arrowprops=dict(facecolor='red', arrowstyle='->'),
                               fontsize=10, color='red')

            # 子图3：累积遗憾的平滑曲线
            axs[1, 0].plot(smoothed_cumulative_regret, marker='o', linestyle='-', color='red', label='Smoothed Cumulative Regret')
            axs[1, 0].set_title('Cumulative Regret (Smoothed)')
            axs[1, 0].set_xlabel('Time Step')
            axs[1, 0].set_ylabel('Cumulative Regret')

            # 标记最大值和最小值
            min_idx = np.argmin(smoothed_cumulative_regret)
            axs[1, 0].annotate(f'Min (x={min_idx}, y={smoothed_cumulative_regret[min_idx]:.2f})',
                               xy=(min_idx, smoothed_cumulative_regret[min_idx]),
                               xytext=(min_idx, smoothed_cumulative_regret[min_idx] + 0.05),
                               arrowprops=dict(facecolor='blue', arrowstyle='->'),
                               fontsize=10, color='blue')

            max_idx = np.argmax(smoothed_cumulative_regret)
            axs[1, 0].annotate(f'Max (x={max_idx}, y={smoothed_cumulative_regret[max_idx]:.2f})',
                               xy=(max_idx, smoothed_cumulative_regret[max_idx]),
                               xytext=(max_idx, smoothed_cumulative_regret[max_idx] - 0.05),
                               arrowprops=dict(facecolor='red', arrowstyle='->'),
                               fontsize=10, color='red')

            # 子图4：Top-k Accuracy
            axs[1, 1].bar(top_k_accuracy.keys(), top_k_accuracy.values(), color='purple')
            axs[1, 1].set_title('Top-k Accuracy')
            axs[1, 1].set_xlabel('k')
            axs[1, 1].set_ylabel('Accuracy')

            # 调整布局
            plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.95)) # 调整 tight_layout，使得 suptitle 不会与子图重叠
            plt.show()
            
        def plot_epsilon_vs_cumulative_regret(self):
            """
            绘制 epsilon vs cumulative regret 图。
            """
            epsilon_values = np.arange(self.e_vs_cr_epsilon_min, self.e_vs_cr_epsilon_max, self.e_vs_cr_epsilon_step)
            cumulative_regrets = []
        
            for epsilon in epsilon_values:
                _, _, _, cumulative_regret, _ = self.algorithm(epsilon)
                cumulative_regrets.append(cumulative_regret[-1])
        
            min_idx = np.argmin(cumulative_regrets)
            max_idx = np.argmax(cumulative_regrets)
        
            plt.figure(figsize=(10, 6))
            plt.plot(epsilon_values, cumulative_regrets, marker='o', linestyle='-', color='blue')
            plt.title('Epsilon vs Cumulative Regret')
            plt.xlabel('Epsilon')
            plt.ylabel('Cumulative Regret')
        
            plt.annotate(f'Min (x={epsilon_values[min_idx]:.3f}, y={cumulative_regrets[min_idx]:.2f})',
                         xy=(epsilon_values[min_idx], cumulative_regrets[min_idx]),
                         xytext=(epsilon_values[min_idx] + 0.02, cumulative_regrets[min_idx] + 0.02),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')
        
            plt.annotate(f'Max (x={epsilon_values[max_idx]:.3f}, y={cumulative_regrets[max_idx]:.2f})',
                         xy=(epsilon_values[max_idx], cumulative_regrets[max_idx]),
                         xytext=(epsilon_values[max_idx] - 0.1, cumulative_regrets[max_idx] - 0.02),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')
        
            plt.grid(True)
            plt.show()

    class AdaptiveEpsilonGreedy:
        """
        Adaptive Epsilon-Greedy 算法类。
        在 Epsilon-Greedy 算法的基础上，根据时间步调整 epsilon 的值。
        越靠后，epsilon 越小，即 epsilon = 1 - t / T_test。
        但是为了保证 epsilon 不会小于最小值，因此设置了一个 min_epsilon 参数，表示算法中可以使用的最小 epsilon 值。
        原先的 epsilon greedy 算法中的 epsilon 在此算法中相当于初始值。
        
        func:
        - algorithm: 运行 adaptive epsilon-greedy 算法，返回选择的节点、每个节点的选择次数、单次遗憾、累积遗憾以及 Top-k Accuracy。
        - plot_results: 绘制 adaptive epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy 以及 epsilon 随时间的变化。
        - plot_epsilon_vs_cumulative_regret: 绘制 epsilon vs cumulative regret 图。
        - dynamic_plot: 动态可视化 adaptive epsilon-greedy 算法的结果，init_epsilon 和 min_epsilon 可以通过滑块调整。
        - run_and_plot: 运行 adaptive epsilon-greedy 算法并绘制结果，用于动态可视化图中的 init_epsilon 和 min_epsilon 可以通过滑块调整。
        """
        def __init__(self, mab):
            self.mab = mab
            self.config_aeg = self.mab.config.adaptive_epsilon_greedy

            self.dynamic_init_epsilon_min = self.config_aeg.dynamic_plot.init_epsilon_min  # 动态 epsilon 的最小值
            self.dynamic_init_epsilon_max = self.config_aeg.dynamic_plot.init_epsilon_max  # 动态 epsilon 的最大值
            self.dynamic_init_epsilon_step = self.config_aeg.dynamic_plot.init_epsilon_step  # 动态 epsilon 的步长
            self.dynamic_init_epsilon_default_value = self.config_aeg.dynamic_plot.init_epsilon_default_value  # 动态 epsilon 的默认值

            self.dynamic_min_epsilon_min = self.config_aeg.dynamic_plot.min_epsilon_min  # 动态 min_epsilon 的最小值
            self.dynamic_min_epsilon_max = self.config_aeg.dynamic_plot.min_epsilon_max  # 动态 min_epsilon 的最大值
            self.dynamic_min_epsilon_step = self.config_aeg.dynamic_plot.min_epsilon_step  # 动态 min_epsilon 的步长
            self.dynamic_min_epsilon_default_value = self.config_aeg.dynamic_plot.min_epsilon_default_value  # 动态 min_epsilon 的默认值

            self.e_vs_cr_epsilon_min = self.config_aeg.epsilon_vs_cumulative_regret.epsilon_min  # epsilon vs cumulative regret 图的 epsilon 最小值
            self.e_vs_cr_epsilon_max = self.config_aeg.epsilon_vs_cumulative_regret.epsilon_max  # epsilon vs cumulative regret 图的 epsilon 最大值
            self.e_vs_cr_epsilon_step = self.config_aeg.epsilon_vs_cumulative_regret.epsilon_step  # epsilon vs cumulative regret 图的 epsilon 步长
            self.e_vs_cr_min_epsilon = self.config_aeg.epsilon_vs_cumulative_regret.min_epsilon  # epsilon vs cumulative regret 图的算法可取到的 epsilon 最小值

        def algorithm(self, init_epsilon: float, min_epsilon: float):
            epsilon = init_epsilon
            def choose_node(epsilon, estimated_means):
                if np.random.rand() < epsilon:
                    return np.random.randint(self.mab.N)
                else:
                    return np.argmax(estimated_means)

            estimated_means = np.zeros(self.mab.N)
            time_counts = np.zeros(self.mab.T_test)
            nodes_counts = np.zeros(self.mab.N)
            single_step_regret = np.zeros(self.mab.T_test)
            epsilon_time = np.zeros(self.mab.T_test)

            for t in range(self.mab.T_test):
                epsilon = max(min_epsilon, epsilon * (1 - t / self.mab.T_test))
                epsilon_time[t] = epsilon
                chosen_node = choose_node(epsilon, estimated_means)
                time_counts[t] = chosen_node
                nodes_counts[chosen_node] += 1
                reward = self.mab.combine_reward[chosen_node, t]
                estimated_means[chosen_node] += (reward - estimated_means[chosen_node]) / nodes_counts[chosen_node]
                single_step_regret[t] = self.mab.combine_reward_optimal_mean - reward

            cumulative_regret = np.cumsum(single_step_regret)
            top_k_accuracy = self.mab.calculate_top_k_accuracy(time_counts, self.mab.top_k)
            return time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, epsilon_time

        def plot_results(self, time_counts: np.ndarray, single_step_regret: np.ndarray, cumulative_regret: np.ndarray, top_k_accuracy: dict, epsilon: float, epsilon_time: np.ndarray) -> None:
            """
            绘制 epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy，以及 epsilon 随时间的变化。
            
            参数:
            - time_counts: 一个数组，表示在每个时间步中选择的节点。
            - single_step_regret: 一个数组，表示每次选择的单次遗憾。
            - cumulative_regret: 一个数组，表示累积遗憾。
            - top_k_accuracy: 一个字典，表示不同 k 值对应的 Top-k Accuracy。
            - epsilon: 当前实验使用的 epsilon 值。
            - epsilon_time: 一个数组，表示每个时间步的 epsilon 值（适用于 adaptive_epsilon_greedy）。
            """
            # 应用平滑方法
            smoothed_single_step_regret = exponential_moving_average(single_step_regret)
            smoothed_cumulative_regret = exponential_moving_average(cumulative_regret)

            fig, axs = plt.subplots(2, 2, figsize=(14, 10))

            fig.suptitle(f"Adaptive Epsilon-Greedy Results\nInitial Epsilon: {epsilon}, Load Reward Method: {self.mab.load_reward_method}, Data Type: {self.mab.data_type}", fontsize=16)

            # 子图1：time_counts 绘制 T_test 次里每次选择的节点
            axs[0, 0].plot(time_counts, marker='o', linestyle='-', color='blue')
            axs[0, 0].set_title('Node Selection Over Time')
            axs[0, 0].set_xlabel('Time Step')
            axs[0, 0].set_ylabel('Chosen Node')

            # 在子图1上绘制 combine_reward_optimal_node 的横向虚线
            optimal_node = self.mab.combine_reward_optimal_node
            axs[0, 0].axhline(y=optimal_node, color='red', linestyle='--', label='Optimal Node')
            axs[0, 0].legend()

            # 子图2：smoothed_single_step_regret 和 smoothed_cumulative_regret 使用双 y 轴
            ax1 = axs[0, 1]
            ax2 = ax1.twinx()

            ax1.plot(smoothed_single_step_regret, marker='o', linestyle='-', color='green', label='Smoothed Single Step Regret')
            ax2.plot(smoothed_cumulative_regret, marker='o', linestyle='-', color='red', label='Smoothed Cumulative Regret')

            ax1.set_title('Single Step and Cumulative Regret (Smoothed)')
            ax1.set_xlabel('Time Step')
            ax1.set_ylabel('Single Step Regret', color='green')
            ax2.set_ylabel('Cumulative Regret', color='red')

            # 标记最小和最大点
            min_idx = np.argmin(smoothed_single_step_regret)
            max_idx = np.argmax(smoothed_single_step_regret)
            ax1.annotate(f'Min (x={min_idx}, y={smoothed_single_step_regret[min_idx]:.2f})',
                         xy=(min_idx, smoothed_single_step_regret[min_idx]),
                         xytext=(min_idx, smoothed_single_step_regret[min_idx] + 0.05),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            ax1.annotate(f'Max (x={max_idx}, y={smoothed_single_step_regret[max_idx]:.2f})',
                         xy=(max_idx, smoothed_single_step_regret[max_idx]),
                         xytext=(max_idx, smoothed_single_step_regret[max_idx] - 0.05),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            # 子图3：绘制 epsilon_time 随时间的变化
            axs[1, 0].plot(epsilon_time, marker='o', linestyle='-', color='purple')
            axs[1, 0].set_title('Epsilon over Time')
            axs[1, 0].set_xlabel('Time Step')
            axs[1, 0].set_ylabel('Epsilon')

            # 子图4：柱状图绘制 top_k_accuracy
            axs[1, 1].bar(top_k_accuracy.keys(), top_k_accuracy.values(), color='purple')
            axs[1, 1].set_title('Top-k Accuracy')
            axs[1, 1].set_xlabel('k')
            axs[1, 1].set_ylabel('Accuracy')

            # 调整布局
            plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.95))  # 调整 tight_layout，使得 suptitle 不会与子图重叠
            plt.show()

        def plot_epsilon_vs_cumulative_regret(self):
            epsilon_values = np.arange(self.e_vs_cr_epsilon_min, self.e_vs_cr_epsilon_max, self.e_vs_cr_epsilon_step)
            cumulative_regrets = []

            for epsilon in epsilon_values:
                _, _, _, cumulative_regret, _, _ = self.algorithm(epsilon, self.e_vs_cr_min_epsilon)
                cumulative_regrets.append(cumulative_regret[-1])

            min_idx = np.argmin(cumulative_regrets)
            max_idx = np.argmax(cumulative_regrets)

            plt.figure(figsize=(10, 6))
            plt.plot(epsilon_values, cumulative_regrets, marker='o', linestyle='-', color='blue')
            plt.title('Epsilon vs Cumulative Regret')
            plt.xlabel('Epsilon')
            plt.ylabel('Cumulative Regret')

            plt.annotate(f'Min (x={epsilon_values[min_idx]:.3f}, y={cumulative_regrets[min_idx]:.2f})',
                         xy=(epsilon_values[min_idx], cumulative_regrets[min_idx]),
                         xytext=(epsilon_values[min_idx] + 0.02, cumulative_regrets[min_idx] + 0.02),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            plt.annotate(f'Max (x={epsilon_values[max_idx]:.3f}, y={cumulative_regrets[max_idx]:.2f})',
                         xy=(epsilon_values[max_idx], cumulative_regrets[max_idx]),
                         xytext=(epsilon_values[max_idx] - 0.1, cumulative_regrets[max_idx] - 0.02),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            plt.grid(True)
            plt.show()


    class DynamicAdaptiveEpsilonGreedy:
        """
        Dynamic Adaptive Epsilon-Greedy 算法类。
        在Adaptive Epsilon-Greedy算法的基础上，根据单步遗憾调整epsilon。
        最初是手动设置阈值，当单步遗憾超过阈值时，增加epsilon，否则减小epsilon。
        （最初始的变化幅度是init_alpha * 上一次的单步遗憾。）
        当前使用的变化幅度是遗憾值与阈值之间的差距，归一化处理后的相对位置。
        但是这种方法不够灵活，因此改为根据单步遗憾的百分位数调整epsilon。
        
        func:
        - algorithm: 运行 dynamic adaptive epsilon-greedy 算法，返回选择的节点、每个节点的选择次数、单次遗憾、累积遗憾以及 Top-k Accuracy。
        - plot_results: 绘制 dynamic adaptive epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy 以及 epsilon 随时间的变化。
        - plot_epsilon_vs_cumulative_regret: 绘制 epsilon vs cumulative regret 图。
        - dynamic_plot: 动态可视化 dynamic adaptive epsilon-greedy 算法的结果
        - run_and_plot: 运行 dynamic adaptive epsilon-greedy 算法并绘制结果，init_epsilon, percentiles 可以通过滑块调整。
        """
        def __init__(self, mab):
            self.mab = mab
            self.config_daeg = self.mab.config.dynamic_adaptive_epsilon_greedy

            self.min_epsilon = self.config_daeg.min_epsilon  # 最小 epsilon 值
            self.max_epsilon = self.config_daeg.max_epsilon  # 最大 epsilon 值
            # self.threshold = self.mab.config.dynamic_adaptive_epsilon_greedy.threshold  # 阈值

            self.dynamic_init_epsilon_min = self.config_daeg.dynamic_plot.init_epsilon_min  # 动态 epsilon 的最小值
            self.dynamic_init_epsilon_max = self.config_daeg.dynamic_plot.init_epsilon_max  # 动态 epsilon 的最大值
            self.dynamic_init_epsilon_step = self.config_daeg.dynamic_plot.init_epsilon_step  # 动态 epsilon 的步长
            self.dynamic_init_epsilon_default_value = self.config_daeg.dynamic_plot.init_epsilon_default_value  # 动态 epsilon 的默认值

            self.dynamic_percentiles_min = self.config_daeg.dynamic_plot.percentiles_min  # 动态 percentiles 的最小值
            self.dynamic_percentiles_max = self.config_daeg.dynamic_plot.percentiles_max  # 动态 percentiles 的最大值
            self.dynamic_percentiles_step = self.config_daeg.dynamic_plot.percentiles_step  # 动态 percentiles 的步长
            self.dynamic_percentiles_default_value = self.config_daeg.dynamic_plot.percentiles_default_value  # 动态 percentiles 的默认值

            self.e_vs_cr_epsilon_min = self.config_daeg.epsilon_vs_cumulative_regret.epsilon_min  # epsilon vs cumulative regret 图的 epsilon 最小值
            self.e_vs_cr_epsilon_max = self.config_daeg.epsilon_vs_cumulative_regret.epsilon_max  # epsilon vs cumulative regret 图的 epsilon 最大值
            self.e_vs_cr_epsilon_step = self.config_daeg.epsilon_vs_cumulative_regret.epsilon_step  # epsilon vs cumulative regret 图的 epsilon 步长
            self.e_vs_cr_percentiles = self.config_daeg.epsilon_vs_cumulative_regret.percentiles  # epsilon vs cumulative regret 图的百分位数

        def algorithm(self, init_epsilon: float, percentiles: float):
            epsilon = init_epsilon
            def choose_node(epsilon, estimated_means):
                if np.random.rand() < epsilon:
                    return np.random.randint(self.mab.N)
                else:
                    return np.argmax(estimated_means)

            estimated_means = np.zeros(self.mab.N)
            time_counts = np.zeros(self.mab.T_test)
            nodes_counts = np.zeros(self.mab.N)
            single_step_regret = np.zeros(self.mab.T_test)
            # cumulative_regret = np.zeros(self.mab.T_test)
            epsilon_time = np.zeros(self.mab.T_test)

            for t in range(self.mab.T_test):
                if t > 0: # 如果 t > 0，则根据单步遗憾调整epsilon，计算百分位数
                    threshold = np.percentile(single_step_regret[:t], percentiles)
                    if single_step_regret[t - 1] > threshold:
                        # 如果单步遗憾值超过了阈值，计算遗憾值与阈值之间的差距，并进行归一化处理。归一化的计算方法是将差值除以阈值（加上一个非常小的值 1e−6，以防止分母为零）。
                        # 归一化的结果 adjustment 会是一个相对值，用来调整 epsilon。这个值越大，说明当前的遗憾值相对于阈值越大，调整的幅度也就越大。
                        adjustment = (single_step_regret[t - 1] - threshold) / (threshold + 1e-6)  # 归一化的相对位置
                        # 当前遗憾值超过阈值的情况下，通过增加 epsilon 的值来增强探索的力度
                        epsilon = min(self.max_epsilon, epsilon + adjustment)
                    else:
                        # 如果当前的单步遗憾没有超过阈值，说明当前的动作表现相对较好，因此倾向于减少探索，更多地利用当前的策略。
                        adjustment = (threshold - single_step_regret[t - 1]) / (threshold + 1e-6)
                        epsilon = max(self.min_epsilon, epsilon - adjustment)
                else:
                    # 如果 t == 0，则不计算百分位数，直接使用初始 epsilon
                    epsilon_time[t] = epsilon

                epsilon_time[t] = epsilon
                chosen_node = choose_node(epsilon, estimated_means)
                time_counts[t] = chosen_node
                nodes_counts[chosen_node] += 1
                reward = self.mab.combine_reward[chosen_node, t]
                estimated_means[chosen_node] += (reward - estimated_means[chosen_node]) / nodes_counts[chosen_node]
                single_step_regret[t] = self.mab.combine_reward_optimal_mean - reward

            cumulative_regret = np.cumsum(single_step_regret)
            top_k_accuracy = self.mab.calculate_top_k_accuracy(time_counts, self.mab.top_k)
            return time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, epsilon_time

        def plot_results(self, time_counts: np.ndarray, single_step_regret: np.ndarray, cumulative_regret: np.ndarray, top_k_accuracy: dict, epsilon: float, epsilon_time: np.ndarray) -> None:
            """
            绘制 epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy，以及 epsilon 随时间的变化。
            
            参数:
            - time_counts: 一个数组，表示在每个时间步中选择的节点。
            - single_step_regret: 一个数组，表示每次选择的单次遗憾。
            - cumulative_regret: 一个数组，表示累积遗憾。
            - top_k_accuracy: 一个字典，表示不同 k 值对应的 Top-k Accuracy。
            - epsilon: 当前实验使用的 epsilon 值。
            - epsilon_time: 一个数组，表示每个时间步的 epsilon 值（适用于 adaptive_epsilon_greedy）。
            """
            # 应用平滑方法
            smoothed_single_step_regret = exponential_moving_average(single_step_regret)
            smoothed_cumulative_regret = exponential_moving_average(cumulative_regret)

            fig, axs = plt.subplots(2, 2, figsize=(14, 10))

            fig.suptitle(f"Dynamically Adaptive Epsilon-Greedy Results\nInitial Epsilon: {epsilon}, Load Reward Method: {self.mab.load_reward_method}, Data Type: {self.mab.data_type}", fontsize=16)

            # 子图1：time_counts 绘制 T_test 次里每次选择的节点
            axs[0, 0].plot(time_counts, marker='o', linestyle='-', color='blue')
            axs[0, 0].set_title('Node Selection Over Time')
            axs[0, 0].set_xlabel('Time Step')
            axs[0, 0].set_ylabel('Chosen Node')

            # 在子图1上绘制 combine_reward_optimal_node 的横向虚线
            optimal_node = self.mab.combine_reward_optimal_node
            axs[0, 0].axhline(y=optimal_node, color='red', linestyle='--', label='Optimal Node')
            axs[0, 0].legend()

            # 子图2：smoothed_single_step_regret 和 smoothed_cumulative_regret 使用双 y 轴
            ax1 = axs[0, 1]
            ax2 = ax1.twinx()

            ax1.plot(smoothed_single_step_regret, marker='o', linestyle='-', color='green', label='Smoothed Single Step Regret')
            ax2.plot(smoothed_cumulative_regret, marker='o', linestyle='-', color='red', label='Smoothed Cumulative Regret')

            ax1.set_title('Single Step and Cumulative Regret (Smoothed)')
            ax1.set_xlabel('Time Step')
            ax1.set_ylabel('Single Step Regret', color='green')
            ax2.set_ylabel('Cumulative Regret', color='red')

            # 标记最小和最大点
            min_idx = np.argmin(smoothed_single_step_regret)
            max_idx = np.argmax(smoothed_single_step_regret)
            ax1.annotate(f'Min (x={min_idx}, y={smoothed_single_step_regret[min_idx]:.2f})',
                         xy=(min_idx, smoothed_single_step_regret[min_idx]),
                         xytext=(min_idx, smoothed_single_step_regret[min_idx] + 0.05),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            ax1.annotate(f'Max (x={max_idx}, y={smoothed_single_step_regret[max_idx]:.2f})',
                         xy=(max_idx, smoothed_single_step_regret[max_idx]),
                         xytext=(max_idx, smoothed_single_step_regret[max_idx] - 0.05),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            # 子图3：绘制 epsilon_time 随时间的变化
            axs[1, 0].plot(epsilon_time, marker='o', linestyle='-', color='purple')
            axs[1, 0].set_title('Epsilon over Time')
            axs[1, 0].set_xlabel('Time Step')
            axs[1, 0].set_ylabel('Epsilon')

            # 子图4：柱状图绘制 top_k_accuracy
            axs[1, 1].bar(top_k_accuracy.keys(), top_k_accuracy.values(), color='purple')
            axs[1, 1].set_title('Top-k Accuracy')
            axs[1, 1].set_xlabel('k')
            axs[1, 1].set_ylabel('Accuracy')

            # 调整布局
            plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.95))  # 调整 tight_layout，使得 suptitle 不会与子图重叠
            plt.show()

        def plot_epsilon_vs_cumulative_regret(self):
            """
            绘制 epsilon vs cumulative regret 图。
            """
            # epsilon_values = np.arange(0.0001, 0.2001, 0.0005)
            epsilon_values = np.arange(self.e_vs_cr_epsilon_min, self.e_vs_cr_epsilon_max, self.e_vs_cr_epsilon_step)

            cumulative_regrets = []

            for epsilon in epsilon_values:
                _, _, _, cumulative_regret, _, _ = self.algorithm(epsilon, self.e_vs_cr_percentiles)
                cumulative_regrets.append(cumulative_regret[-1])

            min_idx = np.argmin(cumulative_regrets)
            max_idx = np.argmax(cumulative_regrets)

            plt.figure(figsize=(10, 6))
            plt.plot(epsilon_values, cumulative_regrets, marker='o', linestyle='-', color='blue')
            plt.title('Epsilon vs Cumulative Regret')
            plt.xlabel('Epsilon')
            plt.ylabel('Cumulative Regret')

            plt.annotate(f'Min (x={epsilon_values[min_idx]:.3f}, y={cumulative_regrets[min_idx]:.2f})',
                         xy=(epsilon_values[min_idx], cumulative_regrets[min_idx]),
                         xytext=(epsilon_values[min_idx] + 0.02, cumulative_regrets[min_idx] + 0.02),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            plt.annotate(f'Max (x={epsilon_values[max_idx]:.3f}, y={cumulative_regrets[max_idx]:.2f})',
                         xy=(epsilon_values[max_idx], cumulative_regrets[max_idx]),
                         xytext=(epsilon_values[max_idx] - 0.1, cumulative_regrets[max_idx] - 0.02),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            plt.grid(True)
            plt.show()


    class Boltzmann:
        """
        Boltzmann 算法类。
        Boltzmann策略基于每个动作的预期奖励（estimated mean）来计算一个概率分布，然后根据这个概率分布选择动作。这个概率分布由每个动作的预期奖励通过温度参数（temperature）调节后得到的软最大值（Softmax函数）确定。
        控制参数为温度。当温度较高时，算法的选择更随机，倾向于探索；当温度较低时，算法更倾向于选择当前已知的最优动作（利用）。
        当温度 τ 很低时，Softmax 函数倾向于将选择概率集中在奖励最高的动作上，类似于贪心策略。
        随着温度 τ 的升高，选择概率变得更加均匀，Softmax 函数逐渐接近于随机选择。
        适用场景：Boltzmann策略在奖励分布相对稳定、噪声较小的场景中表现较好，因为它可以根据已知信息较为精确地调整选择概率。
        
        """
        def __init__(self, mab):
            self.mab = mab
            self.config_boltzmann = self.mab.config.boltzmann

            self.dynamic_temperature_min = self.config_boltzmann.dynamic_plot.temperature_min  # 动态 temperature 的最小值
            self.dynamic_temperature_max = self.config_boltzmann.dynamic_plot.temperature_max  # 动态 temperature 的最大值
            self.dynamic_temperature_step = self.config_boltzmann.dynamic_plot.temperature_step  # 动态 temperature 的步长
            self.dynamic_temperature_default_value = self.config_boltzmann.dynamic_plot.temperature_default_value  # 动态 temperature 的默认值

            self.t_vs_cr_temperature_min = self.config_boltzmann.temperature_vs_cumulative_regret.temperature_min  # temperature vs cumulative regret 图的 temperature 最小值
            self.t_vs_cr_temperature_max = self.config_boltzmann.temperature_vs_cumulative_regret.temperature_max  # temperature vs cumulative regret 图的 temperature 最大值
            self.t_vs_cr_temperature_step = self.config_boltzmann.temperature_vs_cumulative_regret.temperature_step  # temperature vs cumulative regret 图的 temperature 步长

        def algorithm(self, temperature: float):
            def choose_node(temperature: float, estimated_means: np.ndarray):
                exp_values = np.exp(estimated_means / temperature)
                probabilities = exp_values / np.sum(exp_values)
                return np.random.choice(self.mab.N, p=probabilities), probabilities

            estimated_means = np.zeros(self.mab.N)
            time_counts = np.zeros(self.mab.T_test)
            nodes_counts = np.zeros(self.mab.N)
            single_step_regret = np.zeros(self.mab.T_test)
            time_probabilities = np.zeros((self.mab.T_test, self.mab.N))

            for t in range(self.mab.T_test):
                chosen_node, probabilities = choose_node(temperature, estimated_means)
                time_counts[t] = chosen_node
                nodes_counts[chosen_node] += 1
                time_probabilities[t] = probabilities
                reward = self.mab.combine_reward[chosen_node, t]
                estimated_means[chosen_node] += (reward - estimated_means[chosen_node]) / nodes_counts[chosen_node]
                single_step_regret[t] = self.mab.combine_reward_optimal_mean - reward

            cumulative_regret = np.cumsum(single_step_regret)
            top_k_accuracy = self.mab.calculate_top_k_accuracy(time_counts, self.mab.top_k)
            return time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, time_probabilities

        def plot_results(self, time_counts: np.ndarray, single_step_regret: np.ndarray, cumulative_regret: np.ndarray, top_k_accuracy: dict, temperature: float, temperature_time_probabilities: np.ndarray) -> None:
            """
            绘制 epsilon-greedy 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy，以及 epsilon 随时间的变化。
            
            参数:
            - time_counts: 一个数组，表示在每个时间步中选择的节点。
            - single_step_regret: 一个数组，表示每次选择的单次遗憾。
            - cumulative_regret: 一个数组，表示累积遗憾。
            - top_k_accuracy: 一个字典，表示不同 k 值对应的 Top-k Accuracy。
            - temperature: 当前实验使用的温度值。
            - temperature_time_probabilities: 一个二维数组，表示每个时间步的节点选择概率。
            """
            # 应用平滑方法
            smoothed_single_step_regret = exponential_moving_average(single_step_regret)
            smoothed_cumulative_regret = exponential_moving_average(cumulative_regret)

            fig, axs = plt.subplots(2, 2, figsize=(14, 10))
            
            fig.suptitle(f"Boltzmann Results\nTemperature: {temperature}, Load Reward Method: {self.mab.load_reward_method}, Data Type: {self.mab.data_type}", fontsize=16)

            # 子图1：time_counts 绘制 T_test 次里每次选择的节点
            axs[0, 0].plot(time_counts, marker='o', linestyle='-', color='blue')
            axs[0, 0].set_title('Node Selection Over Time')
            axs[0, 0].set_xlabel('Time Step')
            axs[0, 0].set_ylabel('Chosen Node')

            # 在子图1上绘制 combine_reward_optimal_node 的横向虚线
            optimal_node = self.mab.combine_reward_optimal_node
            axs[0, 0].axhline(y=optimal_node, color='red', linestyle='--', label='Optimal Node')
            axs[0, 0].legend()

            # 子图2：smoothed_single_step_regret 和 smoothed_cumulative_regret 使用双 y 轴
            ax1 = axs[0, 1]
            ax2 = ax1.twinx()

            ax1.plot(smoothed_single_step_regret, marker='o', linestyle='-', color='green', label='Smoothed Single Step Regret')
            ax2.plot(smoothed_cumulative_regret, marker='o', linestyle='-', color='red', label='Smoothed Cumulative Regret')

            ax1.set_title('Single Step and Cumulative Regret (Smoothed)')
            ax1.set_xlabel('Time Step')
            ax1.set_ylabel('Single Step Regret', color='green')
            ax2.set_ylabel('Cumulative Regret', color='red')

            # 标记最小和最大点
            min_idx = np.argmin(smoothed_single_step_regret)
            max_idx = np.argmax(smoothed_single_step_regret)
            ax1.annotate(f'Min (x={min_idx}, y={smoothed_single_step_regret[min_idx]:.2f})',
                         xy=(min_idx, smoothed_single_step_regret[min_idx]),
                         xytext=(min_idx, smoothed_single_step_regret[min_idx] + 0.05),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            ax1.annotate(f'Max (x={max_idx}, y={smoothed_single_step_regret[max_idx]:.2f})',
                         xy=(max_idx, smoothed_single_step_regret[max_idx]),
                         xytext=(max_idx, smoothed_single_step_regret[max_idx] - 0.05),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            # 子图3：绘制 temperature_time_probabilities 随时间的变化
            for i in range(self.mab.N):
                axs[1, 0].plot(temperature_time_probabilities[:, i], marker='o', linestyle='-', label=f'Node {i}')
            axs[1, 0].set_title('Node Selection Probabilities Over Time')
            axs[1, 0].set_xlabel('Time Step')
            axs[1, 0].set_ylabel('Probability')
            axs[1, 0].legend()
            
            # 子图4：柱状图绘制 top_k_accuracy
            axs[1, 1].bar(top_k_accuracy.keys(), top_k_accuracy.values(), color='purple')
            axs[1, 1].set_title('Top-k Accuracy')
            axs[1, 1].set_xlabel('k')
            axs[1, 1].set_ylabel('Accuracy')

            # 调整布局
            plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.95))  # 调整 tight_layout，使得 suptitle 不会与子图重叠
            plt.show()
            
        def plot_temperature_vs_cumulative_regret(self):
            """
            绘制 temperature vs cumulative regret 图。
            """
            temperature_values = np.arange(self.t_vs_cr_temperature_min, self.t_vs_cr_temperature_max, self.t_vs_cr_temperature_step)

            cumulative_regrets = []

            for temperature in temperature_values:
                _, _, _, cumulative_regret, _, _ = self.algorithm(temperature)
                cumulative_regrets.append(cumulative_regret[-1])

            min_idx = np.argmin(cumulative_regrets)
            max_idx = np.argmax(cumulative_regrets)

            plt.figure(figsize=(10, 6))
            plt.plot(temperature_values, cumulative_regrets, marker='o', linestyle='-', color='blue')
            plt.title('Temperature vs Cumulative Regret')
            plt.xlabel('Temperature')
            plt.ylabel('Cumulative Regret')

            plt.annotate(f'Min (x={temperature_values[min_idx]:.3f}, y={cumulative_regrets[min_idx]:.2f})',
                         xy=(temperature_values[min_idx], cumulative_regrets[min_idx]),
                         xytext=(temperature_values[min_idx] + 0.02, cumulative_regrets[min_idx] + 0.02),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            plt.annotate(f'Max (x={temperature_values[max_idx]:.3f}, y={cumulative_regrets[max_idx]:.2f})',
                         xy=(temperature_values[max_idx], cumulative_regrets[max_idx]),
                         xytext=(temperature_values[max_idx] - 0.1, cumulative_regrets[max_idx] - 0.02),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            plt.grid(True)
            plt.show()
            
    class UCB:
        """
        UCB 算法类。
        UCB 算法是一种基于置信上界的多臂赌博机算法，通过计算每个动作的置信上界来选择动作。
        UCB 算法的核心思想是：在每个时间步 t，选择使得置信上界最大的动作。
        置信上界的计算方法是：动作的平均奖励 + 置信区间（置信区间的计算方法是置信上界的上限减去置信上界的下限）。
        控制参数为置信区间的参数 c。
        适用场景：UCB 算法在奖励分布相对稳定、噪声较小的场景中表现较好，因为它可以根据已知信息较为精确地调整选择概率。
        """
        def __init__(self, mab):
            self.mab = mab
            self.config_ucb = self.mab.config.ucb

            # self.c = self.config_ucb.c
            self.dynamic_c_min = self.config_ucb.dynamic_plot.c_min  # 动态 c 的最小值
            self.dynamic_c_max = self.config_ucb.dynamic_plot.c_max  # 动态 c 的最大值
            self.dynamic_c_step = self.config_ucb.dynamic_plot.c_step  # 动态 c 的步长
            self.dynamic_c_default_value = self.config_ucb.dynamic_plot.c_default_value  # 动态 c 的默认值
            
            self.c_vs_cr_c_min = self.config_ucb.c_vs_cumulative_regret.c_min  # c vs cumulative regret 图的 c 最小值
            self.c_vs_cr_c_max = self.config_ucb.c_vs_cumulative_regret.c_max  # c vs cumulative regret 图的 c 最大值
            self.c_vs_cr_c_step = self.config_ucb.c_vs_cumulative_regret.c_step  # c vs cumulative regret 图的 c 步长
            
        def algorithm(self, c: float):
            estimated_means = np.zeros(self.mab.N)
            time_counts = np.zeros(self.mab.T_test)
            nodes_counts = np.zeros(self.mab.N)
            single_step_regret = np.zeros(self.mab.T_test)
            time_ucb_values = np.zeros((self.mab.T_test, self.mab.N))

            for t in range(self.mab.T_test):
                ucb_values = estimated_means + c * np.sqrt(np.log(t + 1) / (nodes_counts + 1e-6))
                time_ucb_values[t] = ucb_values
                chosen_node = np.argmax(ucb_values)
                time_counts[t] = chosen_node
                nodes_counts[chosen_node] += 1
                reward = self.mab.combine_reward[chosen_node, t]
                estimated_means[chosen_node] += (reward - estimated_means[chosen_node]) / nodes_counts[chosen_node]
                single_step_regret[t] = self.mab.combine_reward_optimal_mean - reward

            cumulative_regret = np.cumsum(single_step_regret)
            top_k_accuracy = self.mab.calculate_top_k_accuracy(time_counts, self.mab.top_k)
            return time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, time_ucb_values
        
        def plot_results(self, time_counts: np.ndarray, single_step_regret: np.ndarray, cumulative_regret: np.ndarray, top_k_accuracy: dict, c: float, time_ucb_values: np.ndarray) -> None:
            """
            绘制 UCB 算法的结果，包括选择的节点、单次遗憾、累积遗憾、Top-k Accuracy。
            
            参数:
            - time_counts: 一个数组，表示在每个时间步中选择的节点。
            - single_step_regret: 一个数组，表示每次选择的单次遗憾。
            - cumulative_regret: 一个数组，表示累积遗憾。
            - top_k_accuracy: 一个字典，表示不同 k 值对应的 Top-k Accuracy。
            - c: 当前实验使用的置信区间参数。
            """
            # 应用平滑方法
            smoothed_single_step_regret = exponential_moving_average(single_step_regret)
            smoothed_cumulative_regret = exponential_moving_average(cumulative_regret)

            fig, axs = plt.subplots(2, 2, figsize=(14, 10))

            fig.suptitle(f"UCB Results\nConfidence Interval Parameter: {c}, Load Reward Method: {self.mab.load_reward_method}, Data Type: {self.mab.data_type}", fontsize=16)

            # 子图1：time_counts 绘制 T_test 次里每次选择的节点
            axs[0, 0].plot(time_counts, marker='o', linestyle='-', color='blue')
            axs[0, 0].set_title('Node Selection Over Time')
            axs[0, 0].set_xlabel('Time Step')
            axs[0, 0].set_ylabel('Chosen Node')

            # 在子图1上绘制 combine_reward_optimal_node 的横向虚线
            optimal_node = self.mab.combine_reward_optimal_node
            axs[0, 0].axhline(y=optimal_node, color='red', linestyle='--', label='Optimal Node')
            axs[0, 0].legend()

            # 子图2：smoothed_single_step_regret 和 smoothed_cumulative_regret 使用双 y 轴
            ax1 = axs[0, 1]
            ax2 = ax1.twinx()

            ax1.plot(smoothed_single_step_regret, marker='o', linestyle='-', color='green', label='Smoothed Single Step Regret')
            ax2.plot(smoothed_cumulative_regret, marker='o', linestyle='-', color='red', label='Smoothed Cumulative Regret')

            ax1.set_title('Single Step and Cumulative Regret (Smoothed)')
            ax1.set_xlabel('Time Step')
            ax1.set_ylabel('Single Step Regret', color='green')
            ax2.set_ylabel('Cumulative Regret', color='red')

            # 标记最小和最大点
            min_idx = np.argmin(smoothed_single_step_regret)
            max_idx = np.argmax(smoothed_single_step_regret)
            ax1.annotate(f'Min (x={min_idx}, y={smoothed_single_step_regret[min_idx]:.2f})',
                         xy=(min_idx, smoothed_single_step_regret[min_idx]),
                         xytext=(min_idx, smoothed_single_step_regret[min_idx] + 0.05),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            ax1.annotate(f'Max (x={max_idx}, y={smoothed_single_step_regret[max_idx]:.2f})',
                         xy=(max_idx, smoothed_single_step_regret[max_idx]),
                         xytext=(max_idx, smoothed_single_step_regret[max_idx] - 0.05),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            # 子图3：绘制 time_ucb_values 随时间的变化
            for i in range(self.mab.N):
                axs[1, 0].plot(time_ucb_values[:, i], marker='o', linestyle='-', label=f'Node {i}')
            axs[1, 0].set_title('UCB Values Over Time')
            axs[1, 0].set_xlabel('Time Step')
            axs[1, 0].set_ylabel('UCB Value')
            axs[1, 0].legend()
            
            # 子图4：柱状图绘制 top_k_accuracy
            axs[1, 1].bar(top_k_accuracy.keys(), top_k_accuracy.values(), color='purple')
            axs[1, 1].set_title('Top-k Accuracy')
            axs[1, 1].set_xlabel('k')
            axs[1, 1].set_ylabel('Accuracy')

            # 调整布局
            plt.tight_layout(rect=(0.0, 0.0, 1.0, 0.95))  # 调整 tight_layout，使得 suptitle 不会与子图重叠
            plt.show()
            
        def plot_confidence_interval_parameter_vs_cumulative_regret(self):
            """
            绘制 confidence interval parameter vs cumulative regret 图。
            """
            c_values = np.arange(self.c_vs_cr_c_min, self.c_vs_cr_c_max, self.c_vs_cr_c_step)

            cumulative_regrets = []

            for c in c_values:
                _, _, _, cumulative_regret, _, _ = self.algorithm(c)
                cumulative_regrets.append(cumulative_regret[-1])

            min_idx = np.argmin(cumulative_regrets)
            max_idx = np.argmax(cumulative_regrets)

            plt.figure(figsize=(10, 6))
            plt.plot(c_values, cumulative_regrets, marker='o', linestyle='-', color='blue')
            plt.title('Confidence Interval Parameter vs Cumulative Regret')
            plt.xlabel('Confidence Interval Parameter')
            plt.ylabel('Cumulative Regret')

            plt.annotate(f'Min (x={c_values[min_idx]:.1f}, y={cumulative_regrets[min_idx]:.2f})',
                         xy=(c_values[min_idx], cumulative_regrets[min_idx]),
                         xytext=(c_values[min_idx] + 0.1, cumulative_regrets[min_idx] + 0.1),
                         arrowprops=dict(facecolor='blue', arrowstyle='->'),
                         fontsize=10, color='blue')

            plt.annotate(f'Max (x={c_values[max_idx]:.1f}, y={cumulative_regrets[max_idx]:.2f})',
                         xy=(c_values[max_idx], cumulative_regrets[max_idx]),
                         xytext=(c_values[max_idx] - 0.1, cumulative_regrets[max_idx] - 0.1),
                         arrowprops=dict(facecolor='red', arrowstyle='->'),
                         fontsize=10, color='red')

            plt.grid(True)
            plt.show()
            

    def epsilon_greedy_plot_interactive(self, epsilon: float):
        """
        交互式绘制 epsilon-greedy 算法的结果。
        :param epsilon: 固定的全局 epsilon 值
        """
        time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy = self.algorithm.algorithm(epsilon)
        self.algorithm.plot_results(time_counts, single_step_regret, cumulative_regret, top_k_accuracy, epsilon)
    
    def adaptive_epsilon_greedy_plot_interactive(self, init_epsilon: float, min_epsilon: float):
        """
        交互式绘制 adaptive epsilon-greedy 算法的结果。
        :param init_epsilon: 初始 epsilon 值
        :param min_epsilon: 最小 epsilon 值
        """
        time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, epsilon_time = self.algorithm.algorithm(init_epsilon, min_epsilon)
        self.algorithm.plot_results(time_counts, single_step_regret, cumulative_regret, top_k_accuracy, init_epsilon, epsilon_time)
        
    def dynamic_adaptive_epsilon_greedy_plot_interactive(self, init_epsilon: float, percentiles: float):
        """
        交互式绘制 dynamic adaptive epsilon-greedy 算法的结果。
        :param init_epsilon: 初始 epsilon 值
        :param percentiles: 百分位数
        """
        time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, epsilon_time = self.algorithm.algorithm(init_epsilon, percentiles)
        self.algorithm.plot_results(time_counts, single_step_regret, cumulative_regret, top_k_accuracy, init_epsilon, epsilon_time)
        
    def boltzmann_plot_interactive(self, temperature: float):
        """
        交互式绘制 Boltzmann 算法的结果。
        :param temperature: 固定的全局 temperature 值
        """
        time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, time_probabilities = self.algorithm.algorithm(temperature)
        self.algorithm.plot_results(time_counts, single_step_regret, cumulative_regret, top_k_accuracy, temperature, time_probabilities)
        
    def ucb_plot_interactive(self, c: float):
        """
        交互式绘制 UCB 算法的结果。
        :param c: 固定的全局 c 值
        """
        time_counts, nodes_counts, single_step_regret, cumulative_regret, top_k_accuracy, time_ucb_values = self.algorithm.algorithm(c)
        self.algorithm.plot_results(time_counts, single_step_regret, cumulative_regret, top_k_accuracy, c, time_ucb_values)

    def dynamic_plot(self):
        """
        动态绘制算法结果。
        """
        
        # 调整 ToggleButtons 部分的代码
        load_reward_method_widget = widgets.ToggleButtons(
            options=['reward_0', 'reward_1'],
            value='reward_0',
            description='Reward Method:',
            button_style='info',  # 可以设置为 'success', 'info', 'warning', 'danger' 来改变按钮颜色
            layout=widgets.Layout(width='150px', height='auto', margin='10px')  # 调整宽度、高度并增加间距
        )
        
        data_type_widget = widgets.ToggleButtons(
            options=['iid', 'ar1'],
            value='iid',
            description='Data Type:',
            button_style='warning',  # 同样可以设置按钮的样式
            layout=widgets.Layout(width='150px', height='auto', margin='10px')  # 调整宽度、高度并增加间距
        )
        
        algorithm_type_widget = widgets.ToggleButtons(
            options=['epsilon_greedy', 'adaptive_epsilon_greedy', 'dynamic_adaptive_epsilon_greedy', 'boltzmann', 'ucb'],
            value='epsilon_greedy',
            description='Algorithm:',
            button_style='success',  # 设置按钮样式
            style={'button_width': '300px'},  # 使用 style 调整按钮宽度
            layout=widgets.Layout(width='300px', height='auto', margin='10px')  # 设置按钮的宽度
        )
        output = Output()

        def on_button_clicked(b):
            with output:
                output.clear_output()
                self.set_parameters(load_reward_method_widget.value, data_type_widget.value)
                self.plot_combine_rewards()

                # 根据不同的算法类型显示对应的滑块
                if algorithm_type_widget.value == 'epsilon_greedy':
                    self.algorithm = self.EpsilonGreedy(self)
                    epsilon_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_epsilon_min,
                        max=self.algorithm.dynamic_epsilon_max,
                        step=self.algorithm.dynamic_epsilon_step,
                        value=self.algorithm.dynamic_epsilon_default_value,
                        description='Epsilon:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',

                    )
                    interact(self.epsilon_greedy_plot_interactive, epsilon=epsilon_widget)
                    self.algorithm.plot_epsilon_vs_cumulative_regret()
                    
                elif algorithm_type_widget.value == 'adaptive_epsilon_greedy':
                    self.algorithm = self.AdaptiveEpsilonGreedy(self)
                    init_epsilon_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_init_epsilon_min,
                        max=self.algorithm.dynamic_init_epsilon_max,
                        step=self.algorithm.dynamic_init_epsilon_step,
                        value=self.algorithm.dynamic_init_epsilon_default_value,
                        description='Initial Epsilon:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                        
                    )
                    min_epsilon_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_min_epsilon_min,
                        max=self.algorithm.dynamic_min_epsilon_max,
                        step=self.algorithm.dynamic_min_epsilon_step,
                        value=self.algorithm.dynamic_min_epsilon_default_value,
                        description='Min Epsilon:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                    )
                    interact(self.adaptive_epsilon_greedy_plot_interactive, init_epsilon=init_epsilon_widget, min_epsilon=min_epsilon_widget)
                    self.algorithm.plot_epsilon_vs_cumulative_regret()
                    
                elif algorithm_type_widget.value == 'dynamic_adaptive_epsilon_greedy':
                    self.algorithm = self.DynamicAdaptiveEpsilonGreedy(self)
                    init_epsilon_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_init_epsilon_min,
                        max=self.algorithm.dynamic_init_epsilon_max,
                        step=self.algorithm.dynamic_init_epsilon_step,
                        value=self.algorithm.dynamic_init_epsilon_default_value,
                        description='Initial Epsilon:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                    )
                    percentiles_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_percentiles_min,
                        max=self.algorithm.dynamic_percentiles_max,
                        step=self.algorithm.dynamic_percentiles_step,
                        value=self.algorithm.dynamic_percentiles_default_value,
                        description='Percentiles:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                    )
                    interact(self.dynamic_adaptive_epsilon_greedy_plot_interactive, init_epsilon=init_epsilon_widget, percentiles=percentiles_widget)
                    self.algorithm.plot_epsilon_vs_cumulative_regret()
                    
                elif algorithm_type_widget.value == 'boltzmann':
                    self.algorithm = self.Boltzmann(self)
                    temperature_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_temperature_min,
                        max=self.algorithm.dynamic_temperature_max,
                        step=self.algorithm.dynamic_temperature_step,
                        value=self.algorithm.dynamic_temperature_default_value,
                        description='Temperature:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                    )
                    interact(self.boltzmann_plot_interactive, temperature=temperature_widget)
                    self.algorithm.plot_temperature_vs_cumulative_regret()
                    
                elif algorithm_type_widget.value == 'ucb':
                    self.algorithm = self.UCB(self)
                    c_widget = widgets.FloatSlider(
                        min=self.algorithm.dynamic_c_min,
                        max=self.algorithm.dynamic_c_max,
                        step=self.algorithm.dynamic_c_step,
                        value=self.algorithm.dynamic_c_default_value,
                        description='Confidence Interval Parameter:',
                        layout=widgets.Layout(width='600px'),
                        readout_format='.3f',
                    )
                    interact(self.ucb_plot_interactive, c=c_widget)
                    self.algorithm.plot_confidence_interval_parameter_vs_cumulative_regret()

        run_button = widgets.Button(
            description='Set Parameters',
            button_style='primary',  # 设置为 'primary' 风格
            layout=widgets.Layout(width='150px', margin='10px')  # 增加按钮的间距
        )
        run_button.on_click(on_button_clicked)
        
        # 确保输出区域显示
        controls = HBox([load_reward_method_widget, data_type_widget, algorithm_type_widget, run_button])
        display(controls, output)
    

In [51]:
mab = MAB(config, reward_data_manager)
mab.dynamic_plot()


Output()